# Análise das forças internas e externas dos nós da rede de condições psiquiátricas

In [1]:
import pandas as pd
import numpy as np

#Leitura dos dados
psych = pd.read_csv('https://github.com/caiops/IA368X/raw/main/p3/data/processed/network_psych.csv')

In [2]:
#Obtendo os "source nodes", atrelados à coluna j, para o cálculo da força externa
psych_out = psych['j'].value_counts().index
print(psych_out)
#Obtendo os "target nodes", atrelados à coluna i, para o cálculo da força interna
psych_in = psych['i'].value_counts().index
print(psych_in)

Index(['Dem', 'DepO', 'PTSD', 'Per', 'BPD', 'Drg', 'Adj', 'Sui', 'AnxO', 'Dev',
       'Imp', 'Sch', 'ADHD', 'MDD', 'Ch', 'GAD', 'Alc'],
      dtype='object')
Index(['Per', 'Alc', 'Sui', 'ADHD', 'Drg', 'AnxO', 'Dev', 'Ch', 'Imp', 'DepO',
       'Sch', 'BPD', 'MDD', 'Adj', 'PTSD', 'GAD', 'Dem'],
      dtype='object')


Cálculo da força externa.

Para cada nó da lista de "source nodes", filtramos a tabela da rede de condições psiquiátricas de modo a obter apenas as linhas nas quais a coluna j continha o nó em questão, realizando a soma dos valores de _log odds_ e guardando o nome do nó e sua respectiva força externa em uma lista.

Em seguida, a lista foi transformada em um dataframe.

In [3]:
#Obtendo as forças externas de cada nó e guardando em uma lista
psych_os = []
for n in psych_out:
  psych_os.append([n, psych.loc[psych['j'] == n, 'log_odds'].sum()])

In [4]:
#Transformando a lista em um dataframe
psych_out_strength = pd.DataFrame(psych_os, columns = ['Abbr', 'Out-strength'])
psych_out_strength

,Abbr,Out-strength
0,Dem,12.125355
1,DepO,17.509396
2,PTSD,10.529130
3,Per,8.243055
4,BPD,8.508113
5,Drg,9.680665
6,Adj,3.342808
7,Sui,6.424233
8,AnxO,3.465695
9,Dev,6.050017


Cálculo da força interna.

Para cada nó da lista de "target nodes", filtramos a tabela da rede de condições psiquiátricas de modo a obter apenas as linhas nas quais a coluna i continha o nó em questão, realizando a soma dos valores de _log odds_ e guardando o nome do nó e sua respectiva força interna em uma lista.

Em seguida, a lista foi transformada em um dataframe.

In [5]:
#Obtendo as forças internas de cada nó e guardando em uma lista
psych_is = []
for n in psych_in:
  psych_is.append([n, psych.loc[psych['i'] == n, 'log_odds'].sum()])

In [6]:
#Transformando a lista em um dataframe
psych_in_strength = pd.DataFrame(psych_is, columns = ['Abbr', 'In-strength'])
psych_in_strength

,Abbr,In-strength
0,Per,10.397913
1,Alc,6.090909
2,Sui,10.611483
3,ADHD,6.272544
4,Drg,8.041445
5,AnxO,3.830856
6,Dev,8.254053
7,Ch,7.881319
8,Imp,8.021220
9,DepO,4.362302


Por fim, os dois dataframes obtidos foram unidos em um só.

In [7]:
#Juntando os dataframes das forças interna e externa
psych_strength = psych_out_strength.merge(psych_in_strength, on=['Abbr'], how='outer')
psych_strength

,Abbr,Out-strength,In-strength
0,Dem,12.125355,4.298031
1,DepO,17.509396,4.362302
2,PTSD,10.529130,3.622096
3,Per,8.243055,10.397913
4,BPD,8.508113,8.686429
5,Drg,9.680665,8.041445
6,Adj,3.342808,3.003921
7,Sui,6.424233,10.611483
8,AnxO,3.465695,3.830856
9,Dev,6.050017,8.254053


O dataframe gerado foi então salvo como um arquivo .csv.

In [8]:
psych_strength.to_csv('psych_strength.csv', index = False)

Cálculo das forças interna e externa relativas.

In [9]:
#Força externa relativa (força externa dividida pela força interna)
psych_strength['relative_out-strength'] = psych_strength['Out-strength']/psych_strength['In-strength']
#Força interna relativa (força interna dividida pela força externa)
psych_strength['relative_in-strength'] = psych_strength['In-strength']/psych_strength['Out-strength']

In [10]:
#Visualizando a tabela após a adição das colunas com as forças relativas
psych_strength

,Abbr,Out-strength,In-strength,relative_out-strength,relative_in-strength
0,Dem,12.125355,4.298031,2.821142,0.354466
1,DepO,17.509396,4.362302,4.013797,0.249141
2,PTSD,10.529130,3.622096,2.906916,0.344007
3,Per,8.243055,10.397913,0.792761,1.261415
4,BPD,8.508113,8.686429,0.979472,1.020958
5,Drg,9.680665,8.041445,1.203846,0.830671
6,Adj,3.342808,3.003921,1.112815,0.898622
7,Sui,6.424233,10.611483,0.605404,1.651790
8,AnxO,3.465695,3.830856,0.904679,1.105364
9,Dev,6.050017,8.254053,0.732975,1.364303


In [11]:
#Visualizando a tabela das forças dos nós ordenada pela força externa absoluta
psych_strength.sort_values(by = 'Out-strength', ascending = False)

,Abbr,Out-strength,In-strength,relative_out-strength,relative_in-strength
1,DepO,17.509396,4.362302,4.013797,0.249141
0,Dem,12.125355,4.298031,2.821142,0.354466
2,PTSD,10.529130,3.622096,2.906916,0.344007
5,Drg,9.680665,8.041445,1.203846,0.830671
4,BPD,8.508113,8.686429,0.979472,1.020958
11,Sch,8.431723,9.391524,0.897801,1.113832
3,Per,8.243055,10.397913,0.792761,1.261415
7,Sui,6.424233,10.611483,0.605404,1.651790
9,Dev,6.050017,8.254053,0.732975,1.364303
10,Imp,4.535763,8.021220,0.565470,1.768439


In [12]:
#Visualizando a tabela das forças dos nós ordenada pela força externa relativa
psych_strength.sort_values(by = 'relative_out-strength', ascending = False)

,Abbr,Out-strength,In-strength,relative_out-strength,relative_in-strength
1,DepO,17.509396,4.362302,4.013797,0.249141
2,PTSD,10.529130,3.622096,2.906916,0.344007
0,Dem,12.125355,4.298031,2.821142,0.354466
5,Drg,9.680665,8.041445,1.203846,0.830671
6,Adj,3.342808,3.003921,1.112815,0.898622
4,BPD,8.508113,8.686429,0.979472,1.020958
8,AnxO,3.465695,3.830856,0.904679,1.105364
11,Sch,8.431723,9.391524,0.897801,1.113832
3,Per,8.243055,10.397913,0.792761,1.261415
9,Dev,6.050017,8.254053,0.732975,1.364303


In [13]:
#Visualizando a tabela das forças dos nós ordenada pela força interna absoluta
psych_strength.sort_values(by = 'In-strength', ascending = False)

,Abbr,Out-strength,In-strength,relative_out-strength,relative_in-strength
7,Sui,6.424233,10.611483,0.605404,1.651790
3,Per,8.243055,10.397913,0.792761,1.261415
11,Sch,8.431723,9.391524,0.897801,1.113832
4,BPD,8.508113,8.686429,0.979472,1.020958
9,Dev,6.050017,8.254053,0.732975,1.364303
5,Drg,9.680665,8.041445,1.203846,0.830671
10,Imp,4.535763,8.021220,0.565470,1.768439
14,Ch,3.374322,7.881319,0.428142,2.335675
12,ADHD,3.421247,6.272544,0.545432,1.833409
16,Alc,1.312181,6.090909,0.215433,4.641821


In [14]:
#Visualizando a tabela das forças dos nós ordenada pela força interna relativa
psych_strength.sort_values(by = 'relative_in-strength', ascending = False)

,Abbr,Out-strength,In-strength,relative_out-strength,relative_in-strength
16,Alc,1.312181,6.090909,0.215433,4.641821
14,Ch,3.374322,7.881319,0.428142,2.335675
13,MDD,2.169673,4.791395,0.452827,2.208349
15,GAD,1.692552,3.258489,0.519429,1.925193
12,ADHD,3.421247,6.272544,0.545432,1.833409
10,Imp,4.535763,8.021220,0.565470,1.768439
7,Sui,6.424233,10.611483,0.605404,1.651790
9,Dev,6.050017,8.254053,0.732975,1.364303
3,Per,8.243055,10.397913,0.792761,1.261415
11,Sch,8.431723,9.391524,0.897801,1.113832
